In [1]:
#Importing the required modules
import nltk
import re
from nltk.corpus import stopwords
import stanza
import pandas as pd
import numpy as np
import collections
import spacy

In [2]:
#making data frame from the dataset 
#This will be the train dataset, scraped from Indeed's website
df = pd.read_csv('Rated_JobPostings_cleaned.csv')
df.describe()

,Unnamed: 0,Low Vision,Hearing Impaired
count,204.000000,204.000000,204.000000
mean,108.352941,1.700980,2.916667
std,62.585689,1.233282,1.571895
min,0.000000,0.000000,0.000000
25%,56.750000,1.000000,2.000000
50%,109.500000,2.000000,3.000000
75%,160.250000,3.000000,4.000000
max,217.000000,5.000000,5.000000


In [3]:
#dropping empty rows
df.dropna()

,Unnamed: 0,title,company name,link,date listed,salary,job_description,Low Vision,Hearing Impaired
0,0,Call Centre Agent,RMA CONTRACTS PTE. LTD.,sg.indeed.com/rc/clk?jk=a833dcac9b845238&fccid...,Posted8 days ago,"$2,600 - $2,800 a month",Handle a large volume of outbound calls in a t...,2,0
1,1,Machine Operator - General Worker,EURO-LINK CONSULTANTS,sg.indeed.com/rc/clk?jk=86d2c472c28278ef&fccid...,Posted7 days ago,"$2,900 - $3,800 a month","6 days work week, off on Sundays.\nWilling to ...",2,5
2,2,Packing,HOT SPICE CATERING PTE. LTD.,sg.indeed.com/rc/clk?jk=60532769fe7fed06&fccid...,Posted30+ days ago,"$5,000 - $6,500 a month",Keep Rice on the Rice steamers on the stipulat...,3,3
3,3,Work-from-home Accountants,Smaths Group of Companies pte ltd,sg.indeed.com/rc/clk?jk=5bb77b27c75a52dc&fccid...,Posted30+ days ago,$15 an hour,Work-from-home Accountants Job Scope? To compl...,0,5
4,4,Cat Care Assistant,PAYMHS PTE LTD,sg.indeed.com/company/PAYMHS-PTE-LTD/jobs/Cat-...,EmployerActive 1 day ago,$10 an hour,"Job Types: Part-time, Permanent, Freelance.",3,3
...,...,...,...,...,...,...,...,...,...
199,212,Human Resource executive,ATO Partners Pte Ltd,sg.indeed.com/company/ATO-Partners-Pte-Ltd/job...,Posted3 days ago,"$1,710 - $4,781 a month",Source and pre-qualify candidates that match b...,2,3
200,213,Hotel / Service Apartment Management Trainee,SEE MY SOLUTIONS,sg.indeed.com/rc/clk?jk=e2eb0bd74612212c&fccid...,Posted10 days ago,"$2,500 - $5,000 a month",We are an outsource company. Looking for Talen...,2,4
201,214,Shop Sales Assistant,DOLISH NAIL SPA,sg.indeed.com/rc/clk?jk=325f4c35f76f8954&fccid...,Posted4 days ago,"$2,900 - $3,400 a month",To ensure product are well stocked and follow ...,3,4
202,215,Chairman Personal Assistant,STRAITDEER PTE. LTD.,sg.indeed.com/rc/clk?jk=3e4c5d33b8f34643&fccid...,Posted12 days ago,"$7,000 - $13,000 a month",Proven ability to work well with all levels of...,1,1


In [4]:
desc_words_list = ['']

for value in df.job_description.str.split(' '):
     desc_words_list.extend(value)
    
print(desc_words_list)


['', 'Handle', 'a', 'large', 'volume', 'of', 'outbound', 'calls', 'in', 'a', 'timely', 'manner.\nEnsure', 'high', 'completion', 'rate', 'of', 'survey', 'for', 'total', 'calls', 'made.\nPerform', 'verification', 'of', 'inaccurate', 'data.', '6', 'days', 'work', 'week,', 'off', 'on', 'Sundays.\nWilling', 'to', 'learn,', 'no', 'experience', 'needed.\nWilling', 'to', 'work', '12', 'hours', 'shift.\nSalary', 'will', 'be', 'paid', 'twice', 'a', 'month.', 'Keep', 'Rice', 'on', 'the', 'Rice', 'steamers', 'on', 'the', 'stipulated', 'timings', 'as', 'per', 'cook', 'advice.\nEnsure', 'proper', 'food', 'delviery', 'without', 'any', 'leakage', 'of', 'the', 'food', 'items.', 'Work-from-home', 'Accountants', 'Job', 'Scope?', 'To', 'complete', 'book-keeping', 'duties', 'off-premises.', 'Pay?', '$15/hour', 'onwards,', 'depending', 'on', 'speed', 'and', 'quality', 'of', 'Jobs', 'completed.…', 'Job', 'Types:', 'Part-time,', 'Permanent,', 'Freelance.', 'Booking', 'of', 'any', 'flights', '/', 'hotels', 'fo

In [5]:
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

def cleanPosting(String:str)->list:
    text = String
    sw_nltk = stopwords.words('english')
    delimiters = ".",";"," ",":","(",")","'",'"',"{","}","[","]","|","\\","/","\n"
    regexPattern = '|'.join(map(re.escape, delimiters))
    words = [word.lower() for word in re.split(regexPattern,text) if ((word.lower() not in sw_nltk) and (word.isalpha())and(len(word)>1))]
    return words

In [6]:
#cleaning the description words
desc_words_string = listToString(desc_words_list)
desc_words = cleanPosting(desc_words_string)

#removing repeated words
desc_words = list(dict.fromkeys(desc_words))
print(desc_words)

['handle', 'large', 'volume', 'outbound', 'calls', 'timely', 'manner', 'ensure', 'high', 'completion', 'rate', 'survey', 'total', 'made', 'perform', 'verification', 'inaccurate', 'data', 'days', 'work', 'sundays', 'willing', 'experience', 'needed', 'hours', 'shift', 'salary', 'paid', 'twice', 'month', 'keep', 'rice', 'steamers', 'stipulated', 'timings', 'per', 'cook', 'advice', 'proper', 'food', 'delviery', 'without', 'leakage', 'items', 'accountants', 'job', 'complete', 'duties', 'hour', 'depending', 'speed', 'quality', 'jobs', 'completed', 'types', 'freelance', 'booking', 'flights', 'hotels', 'staff', 'required', 'office', 'manage', 'coordinate', 'daily', 'general', 'administration', 'permanent', 'send', 'request', 'quote', 'review', 'quotes', 'select', 'competitive', 'bids', 'issue', 'po', 'placed', 'super', 'clustered', 'project', 'meaning', 'cover', 'block', 'hdb', 'need', 'sorting', 'warehouse', 'answer', 'phones', 'direct', 'appropriate', 'team', 'member', 'accurate', 'entry', '

In [7]:
#lemmatization
desc_wordsString = listToString(desc_words)

sp = spacy.load('en_core_web_sm')

text = sp(desc_wordsString)

desc_words = []
for word in text:
    desc_words.append(word.lemma_)
    
print(desc_words)

['handle', 'large', 'volume', 'outbound', 'call', 'timely', 'manner', 'ensure', 'high', 'completion', 'rate', 'survey', 'total', 'make', 'perform', 'verification', 'inaccurate', 'datum', 'day', 'work', 'sunday', 'willing', 'experience', 'need', 'hour', 'shift', 'salary', 'pay', 'twice', 'month', 'keep', 'rice', 'steamer', 'stipulate', 'timing', 'per', 'cook', 'advice', 'proper', 'food', 'delviery', 'without', 'leakage', 'item', 'accountant', 'job', 'complete', 'duty', 'hour', 'depend', 'speed', 'quality', 'job', 'complete', 'type', 'freelance', 'book', 'flight', 'hotel', 'staff', 'require', 'office', 'manage', 'coordinate', 'daily', 'general', 'administration', 'permanent', 'send', 'request', 'quote', 'review', 'quote', 'select', 'competitive', 'bid', 'issue', 'po', 'place', 'super', 'cluster', 'project', 'mean', 'cover', 'block', 'hdb', 'need', 'sort', 'warehouse', 'answer', 'phone', 'direct', 'appropriate', 'team', 'member', 'accurate', 'entry', 'skill', 'account', 'payable', 'provid

In [8]:
#Title words
title_words_list = ['']

for value in df.title.str.split(' '):
     title_words_list.extend(value)
print(title_words_list)

['', 'Call', 'Centre', 'Agent', 'Machine', 'Operator', '-', 'General', 'Worker', 'Packing', 'Work-from-home', 'Accountants', 'Cat', 'Care', 'Assistant', 'Admin,', 'Office', '(5-Day', 'Perm)', 'Packer', 'or', 'Agri-Artisan', '-', 'Join', 'us!', 'Purchaser', 'cum', 'Admin', 'Assistant', 'Parcel', 'Delivery', 'Weekend', 'Admin', 'Data', 'Entry', 'Clerk', 'Wanted', 'Administrative', 'Assistant', 'Waiter', '/', 'Waitress', 'PICKER', '&', 'PACKERS', 'Weekend', 'Data', 'Entry', 'Positions', 'With', 'Good', 'Pay', 'Aircraft', 'Planner', 'Administrative', 'Assistant', 'Aircraft', 'Technician', 'Dish', 'Washer', 'ADMINISTRATOR', 'Home-Based', 'Video', 'Editors', 'Data', 'Entry', 'Executive', 'Hotel', 'Receiptionist', 'Retail', 'Assistant', 'Self', '-', 'Photo', 'Studio', 'Staff', 'Parcel', 'Delivery', 'wallpaper', 'installer', 'Administration', 'Clerk', 'Warehouse', '/', 'Packer', 'Assistant', 'Customer', 'Experience', 'Manager', 'Administrative', 'Officer', 'SCHOOL', 'ADMIN', 'ASSISTANT', 'Part

In [9]:
#cleaning job title words 
title_words_string = listToString(title_words_list)
title_words = cleanPosting(title_words_string)

#removing repeated words
desc_words = list(dict.fromkeys(title_words))
print(title_words)

['call', 'centre', 'agent', 'machine', 'operator', 'general', 'worker', 'packing', 'accountants', 'cat', 'care', 'assistant', 'office', 'perm', 'packer', 'join', 'purchaser', 'cum', 'admin', 'assistant', 'parcel', 'delivery', 'weekend', 'admin', 'data', 'entry', 'clerk', 'wanted', 'administrative', 'assistant', 'waiter', 'waitress', 'picker', 'packers', 'weekend', 'data', 'entry', 'positions', 'good', 'pay', 'aircraft', 'planner', 'administrative', 'assistant', 'aircraft', 'technician', 'dish', 'washer', 'administrator', 'video', 'editors', 'data', 'entry', 'executive', 'hotel', 'receiptionist', 'retail', 'assistant', 'self', 'photo', 'studio', 'staff', 'parcel', 'delivery', 'wallpaper', 'installer', 'administration', 'clerk', 'warehouse', 'packer', 'assistant', 'customer', 'experience', 'manager', 'administrative', 'officer', 'school', 'admin', 'assistant', 'corporate', 'workshop', 'trainer', 'boarding', 'officer', 'warehouse', 'packer', 'cashier', 'library', 'assistant', 'part', 'tim

In [10]:
#lemmatization
title_wordsString = listToString(title_words)

sp = spacy.load('en_core_web_sm')

text = sp(title_wordsString)

title_words = []
for word in text:
    title_words.append(word.lemma_)
    
print(title_words)

['call', 'centre', 'agent', 'machine', 'operator', 'general', 'worker', 'packing', 'accountant', 'cat', 'care', 'assistant', 'office', 'perm', 'packer', 'join', 'purchaser', 'cum', 'admin', 'assistant', 'parcel', 'delivery', 'weekend', 'admin', 'data', 'entry', 'clerk', 'want', 'administrative', 'assistant', 'waiter', 'waitress', 'picker', 'packer', 'weekend', 'data', 'entry', 'position', 'good', 'pay', 'aircraft', 'planner', 'administrative', 'assistant', 'aircraft', 'technician', 'dish', 'washer', 'administrator', 'video', 'editor', 'datum', 'entry', 'executive', 'hotel', 'receiptionist', 'retail', 'assistant', 'self', 'photo', 'studio', 'staff', 'parcel', 'delivery', 'wallpaper', 'installer', 'administration', 'clerk', 'warehouse', 'packer', 'assistant', 'customer', 'experience', 'manager', 'administrative', 'officer', 'school', 'admin', 'assistant', 'corporate', 'workshop', 'trainer', 'boarding', 'officer', 'warehouse', 'packer', 'cashier', 'library', 'assistant', 'part', 'time', '

In [11]:
#Implementing Bag of Words 

#number of words in the vocab
len_desc_vector = len(desc_words)
len_title_vector = len(title_words)

#Index dictionary to assign an index to each word in vocabulary
index_desc_word = {}
index_title_word = {}
i = 0
for word in desc_words:
    index_desc_word[word] = i 
    i += 1

i = 0
for word in title_words:
    index_title_word[word] = i 
    i += 1 


In [12]:
def bag_of_words(sent):
    count_dict = defaultdict(int)
    vec = np.zeros(len_vector)
    for item in sent:
        count_dict[item] += 1
    for key,item in count_dict.items():
        vec[index_word[key]] = item
    return vec 

In [13]:
df = df.reset_index()  # make sure indexes pair with number of rows
desc_sent = []
for index, row in df.iterrows():
    desc_sent.append(row['job_description'])
print(desc_sent)

['Handle a large volume of outbound calls in a timely manner.\nEnsure high completion rate of survey for total calls made.\nPerform verification of inaccurate data.', '6 days work week, off on Sundays.\nWilling to learn, no experience needed.\nWilling to work 12 hours shift.\nSalary will be paid twice a month.', 'Keep Rice on the Rice steamers on the stipulated timings as per cook advice.\nEnsure proper food delviery without any leakage of the food items.', 'Work-from-home Accountants Job Scope? To complete book-keeping duties off-premises. Pay? $15/hour onwards, depending on speed and quality of Jobs completed.…', 'Job Types: Part-time, Permanent, Freelance.', 'Booking of any flights / hotels for staff as per required.\n*CENTRAL | 5-DAY OFFICE HOURS*.\nManage and coordinate daily general office administration duties.', 'Job Types: Full-time, Part-time, Temporary, Permanent.', 'Send request for quote to supplier, review the quotes & select the most competitive bids.\nIssue and manage P

In [14]:
#cleaning the text for inclusion in the vocabulary of Bag of Words
#Replaced individually because when it was clubbed, Jupyter was throwing errors

def sent_cleaning(desc_sent):
    desc_sent_cleaned = []
    for sent in desc_sent:
        desc_sent_cleaned.append(sent.replace("\n", ""))

    characters = "|()"
    desc_sent = []

    for sent in desc_sent_cleaned:
        desc_sent.append(sent.replace("|",""))

    desc_sent_cleaned = []
    for sent in desc_sent:
        desc_sent_cleaned.append(sent.replace("*", ""))

    desc_sent = []

    for sent in desc_sent_cleaned:
        desc_sent.append(sent.replace("(",""))

    desc_sent_cleaned = []
    for sent in desc_sent:
        desc_sent_cleaned.append(sent.replace(")", ""))

    desc_sent = []

    for sent in desc_sent_cleaned:
        desc_sent.append(sent.replace("."," "))

    desc_sent_cleaned = []
    for sent in desc_sent:
        desc_sent_cleaned.append(sent.replace(":", ""))

    desc_sent = []
    for sent in desc_sent_cleaned:
        desc_sent.append(sent.replace(","," "))

    desc_sent_cleaned = []
    for sent in desc_sent:
        desc_sent_cleaned.append(sent.replace("/", ""))

    desc_sent = []
    for sent in desc_sent_cleaned:
        desc_sent.append(sent.replace("&"," "))

    desc_sent_cleaned = []
    for sent in desc_sent:
        desc_sent_cleaned.append(sent.replace("?", ""))

    desc_sent = []
    for sent in desc_sent_cleaned:
        desc_sent.append(sent.replace("…"," "))

    desc_sent_cleaned = []
    for sent in desc_sent:
        desc_sent_cleaned.append(sent.replace("$", ""))
    
    return desc_sent_cleaned

desc_sent_cleaned = sent_cleaning(desc_sent)
print(desc_sent_cleaned)

['Handle a large volume of outbound calls in a timely manner Ensure high completion rate of survey for total calls made Perform verification of inaccurate data ', '6 days work week  off on Sundays Willing to learn  no experience needed Willing to work 12 hours shift Salary will be paid twice a month ', 'Keep Rice on the Rice steamers on the stipulated timings as per cook advice Ensure proper food delviery without any leakage of the food items ', 'Work-from-home Accountants Job Scope To complete book-keeping duties off-premises  Pay 15hour onwards  depending on speed and quality of Jobs completed  ', 'Job Types Part-time  Permanent  Freelance ', 'Booking of any flights  hotels for staff as per required CENTRAL  5-DAY OFFICE HOURS Manage and coordinate daily general office administration duties ', 'Job Types Full-time  Part-time  Temporary  Permanent ', 'Send request for quote to supplier  review the quotes   select the most competitive bids Issue and manage PO  DO  Inv Ensure all PO are

In [15]:
def remove_digits(list):
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list]
    return list

In [16]:
for i in range(len(desc_sent_cleaned)):
    desc_sent_cleaned[i] = desc_sent_cleaned[i].lower()

desc_sent = remove_digits(desc_sent_cleaned)    
    
print(desc_sent)

['handle a large volume of outbound calls in a timely manner ensure high completion rate of survey for total calls made perform verification of inaccurate data ', ' days work week  off on sundays willing to learn  no experience needed willing to work  hours shift salary will be paid twice a month ', 'keep rice on the rice steamers on the stipulated timings as per cook advice ensure proper food delviery without any leakage of the food items ', 'work-from-home accountants job scope to complete book-keeping duties off-premises  pay hour onwards  depending on speed and quality of jobs completed  ', 'job types part-time  permanent  freelance ', 'booking of any flights  hotels for staff as per required central  -day office hours manage and coordinate daily general office administration duties ', 'job types full-time  part-time  temporary  permanent ', 'send request for quote to supplier  review the quotes   select the most competitive bids issue and manage po  do  inv ensure all po are place

In [18]:
#Feature Extraction using TF-IFD method, using TfidfVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

train = desc_sent

# create the transform
vectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

# tokenize and build vocab
vectorizer.fit_transform(train)

# summarize
print(sorted(vectorizer.vocabulary_))

# encode document
vector = vectorizer.transform(train)

['_able', '_assist', '_bringing', 'ability', 'able', 'absence', 'absentees', 'accordance', 'according', 'accordingly', 'account', 'accountants', 'accounting', 'accounts', 'accuracy', 'accurate', 'accurately', 'achieve', 'acrobat', 'actively', 'activities', 'activity', 'addicted', 'additional', 'address', 'adherence', 'admin', 'administer', 'administers', 'administration', 'administrative', 'adobe', 'advanced', 'advantage', 'advantageous', 'advantange', 'advice', 'advising', 'affected', 'afternoon', 'aged', 'agency', 'agent', 'air', 'aircraft', 'airport', 'alert', 'allow', 'allowance', 'allowances', 'ambassador', 'analyse', 'analysis', 'analyze', 'andor', 'animation', 'annual', 'answer', 'answering', 'app', 'applicants', 'applications', 'apply', 'appointment', 'appointments', 'appropriate', 'area', 'areas', 'arrange', 'arrangement', 'arranging', 'arrival', 'art', 'asked', 'aspects', 'assemble', 'assembly', 'asset', 'assigned', 'assist', 'assistance', 'assistant', 'assistants', 'assistin

In [19]:
#Predicting Using Stochastic Gradient Descent Regression
from sklearn.linear_model import SGDRegressor
vi_train_targets = df['Low Vision'].tolist()
vi_regr = SGDRegressor()
vi_regr.fit(vector,vi_train_targets)

SGDRegressor()

In [20]:
#Uploading test Data


test_df = pd.read_csv("Test_Data.csv")
test_df.head()

,Unnamed: 0,title,company name,link,date listed,salary,job description
0,539,Warehouse Assistant,3M,sg.indeed.com/rc/clk?jk=88d02a61607509de&fccid...,Posted30+ days ago,NaN,Support the day-to-day warehouse operations.\n...
1,540,IKEA Tampines - Cashier (Part-Time),IKANO Southeast Asia (IKEA Tampines),sg.indeed.com/company/IKANO-Southeast-Asia-(IK...,EmployerActive 6 days ago,NaN,Part-time hours: 15 per week.\nPart-time hours...
2,541,Office Manager,Forrester Research,sg.indeed.com/rc/clk?jk=a58094101e8465d3&fccid...,Posted18 days ago,NaN,At least one year of post-education experience...
3,542,Customer Service Officer,HYPERSCAL SOLUTIONS PTE. LTD.,sg.indeed.com/rc/clk?jk=3f094bad8567f490&fccid...,Posted4 days ago,"$2,000 - $2,900 a month","As a Customer Service Officer, you will be amb..."
4,543,Food and Beverage Assistant,EGA Wellness,sg.indeed.com/company/Ega-wellness/jobs/Food-B...,Posted1 day ago,"$2,000 - $2,500 a month","No experience required, working hours 9am-6pm...."


In [25]:
#Running the model and predicting

test_df['Low Vision']=[0]*len(test_df)
predictions = []
for index, row in test_df.iterrows():
    test_desc = row['job description']
    test_desc = cleanPosting(test_desc)
    
    #lemmatization
    test_descString = listToString(test_desc)

    sp = spacy.load('en_core_web_sm')

    text = sp(test_descString)

    test_desc = []
    for word in text:
        test_desc.append(word.lemma_)

    test_descString = listToString(test_desc)
    test_desc = []
    test_desc.append(test_descString)
    
    test = test_desc
    tfidf_term_vectors  = vectorizer.transform(test)
    vi_prediction = vi_regr.predict(tfidf_term_vectors)
    
    predictions.append(np.asscalar(vi_prediction))
    
print(predictions)

/home/dell/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


[1.5664315123522523, 0.989807952237365, 1.3438388590737043, 1.3530202359348178, 1.8129275029367276, 1.4717111088632948, 0.9841101681183798, 1.2209554410421024, 0.9214867824744954, 2.123555047425126, 1.3339892419848527, 0.9710728319858052, 0.9619674685268605, 1.2673869585249786, 1.2653543057391188, 1.5590309072885145, 1.2338488140019521, 1.0248701571858176, 1.6764530060407696, 0.9455785082809162, 1.7244456801194472, 2.0908425125226824, 0.44719954384612925, 1.0809227283914278, 1.9089067323881466, 1.6650076740827986, 1.0907539793723897, 1.189200896890176, 1.1293057122237085, 1.3521478143020558, 0.9717926141591198, 0.9309450431809028, 1.3553167266983255, 1.4455687115221885, 1.2330821763832605, 1.0295544954052929, 0.9010350269918646, 1.4082347234572477, 0.9465842032103412, 0.8726573006229863, 0.8609679587399142, 1.111323923992817, 1.0406752977776992, 1.3828024668005643, 1.0233590161679755, 1.1024595021289956, 1.1939953525282578, 1.2255858670007833, 1.5349829088926215, 1.6498816031830983, 1.

In [31]:
#Exporting the dataframe as excel file
pred_df = pd.DataFrame(predictions)
pred_df.head()
pred_df.to_excel('Prediction for Low Vision.xlsx')

In [29]:
#Including the results in the TestDataFrame
frames = [test_df, pred_df]
result_df = pd.concat(frames, axis=1)
result_df.head()

,Unnamed: 0,title,company name,link,date listed,salary,job description,Low Vision,0
0,539,Warehouse Assistant,3M,sg.indeed.com/rc/clk?jk=88d02a61607509de&fccid...,Posted30+ days ago,NaN,Support the day-to-day warehouse operations.\n...,0,1.566432
1,540,IKEA Tampines - Cashier (Part-Time),IKANO Southeast Asia (IKEA Tampines),sg.indeed.com/company/IKANO-Southeast-Asia-(IK...,EmployerActive 6 days ago,NaN,Part-time hours: 15 per week.\nPart-time hours...,0,0.989808
2,541,Office Manager,Forrester Research,sg.indeed.com/rc/clk?jk=a58094101e8465d3&fccid...,Posted18 days ago,NaN,At least one year of post-education experience...,0,1.343839
3,542,Customer Service Officer,HYPERSCAL SOLUTIONS PTE. LTD.,sg.indeed.com/rc/clk?jk=3f094bad8567f490&fccid...,Posted4 days ago,"$2,000 - $2,900 a month","As a Customer Service Officer, you will be amb...",0,1.353020
4,543,Food and Beverage Assistant,EGA Wellness,sg.indeed.com/company/Ega-wellness/jobs/Food-B...,Posted1 day ago,"$2,000 - $2,500 a month","No experience required, working hours 9am-6pm....",0,1.812928
